In [1]:
import os
import numpy as np
import keras
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array,ImageDataGenerator
from PIL import Image
from keras.utils import to_categorical
from keras.models import Sequential, Model,load_model,model_from_json
from keras.optimizers import SGD,Adam
from keras.layers import *
import tensorflow as tf
import matplotlib.pyplot as plt

C:\Users\cpiyu\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def preprocess(image_path):
    img = Image.open(image_path)
    img = img.resize((96,96), Image.ANTIALIAS)
    img = img_to_array(img)
    img = (img - 128) / 128
    return img

In [3]:
def list2np(l, size):
    n = np.array(l)
    return(n.reshape((size,)+img_shape))

In [4]:
data = list()
class_labels = list()
labels = ['Parasitized','Uninfected']
count = 0

while(count<2):
    for image in os.listdir('../cell_images/cell_images/'+labels[count]):
        if(image.split('.')[1] == 'png'):
            img = preprocess('../cell_images/cell_images/'+labels[count]+'/'+image)
            data.append(img)
            class_labels.append(count)
    count = count+1
data = np.array(data)
class_labels = np.array(class_labels)

In [5]:
print(type(data[0]),data[0].shape)
print(type(class_labels))

<class 'numpy.ndarray'> (96, 96, 3)
<class 'numpy.ndarray'>


In [6]:
data.shape,class_labels.shape

((27558, 96, 96, 3), (27558,))

In [7]:
X_train,X_val,y_train,y_val = train_test_split(data,class_labels,shuffle=True,test_size=0.2,random_state=42)
y_train = to_categorical(y_train, num_classes=2)
y_val= to_categorical(y_val, num_classes=2)

In [8]:
model = Sequential()
model.add(Conv2D(32, (3,3), padding="same", input_shape=(96,96,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(32, (3,3), padding="same",activation='relu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Flatten())
model.add(Dense(128,activation='relu'))

model.add(Dense(2,activation='sigmoid'))

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 10, 10, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               409728    
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 258       
Total para

In [10]:
opt = Adam(lr=0.0002, beta_1=0.5,clipvalue = 1)
model.compile(loss='binary_crossentropy', optimizer=opt,metrics=['accuracy'])
model_history = model.fit(X_train,y_train,validation_data=(X_val,y_val),batch_size=128,epochs=25,validation_split=.1,verbose=2)

Train on 22046 samples, validate on 5512 samples
Epoch 1/25
 - 213s - loss: 0.5653 - acc: 0.7060 - val_loss: 0.5131 - val_acc: 0.7336
Epoch 2/25
 - 30s - loss: 0.4025 - acc: 0.8225 - val_loss: 0.3484 - val_acc: 0.8374
Epoch 3/25
 - 29s - loss: 0.2636 - acc: 0.8991 - val_loss: 0.2350 - val_acc: 0.9190
Epoch 4/25
 - 17s - loss: 0.1977 - acc: 0.9284 - val_loss: 0.1827 - val_acc: 0.9400
Epoch 5/25
 - 10s - loss: 0.1682 - acc: 0.9408 - val_loss: 0.1644 - val_acc: 0.9427
Epoch 6/25
 - 10s - loss: 0.1530 - acc: 0.9462 - val_loss: 0.1686 - val_acc: 0.9376
Epoch 7/25
 - 10s - loss: 0.1444 - acc: 0.9496 - val_loss: 0.1772 - val_acc: 0.9327
Epoch 8/25
 - 10s - loss: 0.1358 - acc: 0.9527 - val_loss: 0.1503 - val_acc: 0.9503
Epoch 9/25
 - 10s - loss: 0.1290 - acc: 0.9551 - val_loss: 0.1648 - val_acc: 0.9429
Epoch 10/25
 - 10s - loss: 0.1241 - acc: 0.9569 - val_loss: 0.1423 - val_acc: 0.9502
Epoch 11/25
 - 10s - loss: 0.1188 - acc: 0.9586 - val_loss: 0.1359 - val_acc: 0.9537
Epoch 12/25
 - 10s - los